In [1]:
import wilds
import os
import torchvision.transforms as transforms

In [2]:
# `data` is at the outer project directory 
os.chdir("..")

In [3]:
os.getcwd()

'/storage/ice1/5/4/rso31/meadow'

In [4]:
dataset_lb = wilds.get_dataset(dataset="iwildcam")

In [5]:
len(dataset_lb)

203029

In [6]:
train_lb_data = dataset_lb.get_subset(
    "train",
    transform=transforms.Compose(
    [transforms.Resize((448, 448)), transforms.ToTensor()]
    )
)

In [7]:
test_lb_data = dataset_lb.get_subset(
    "test",
    transform=transforms.Compose(
    [transforms.Resize((448, 448)), transforms.ToTensor()]
    )
)

In [8]:
val_lb_data = dataset_lb.get_subset(
    "val",
    transform=transforms.Compose(
    [transforms.Resize((448, 448)), transforms.ToTensor()]
    )
)

In [9]:
len(train_lb_data)

129809

In [10]:
len(test_lb_data)

42791

In [11]:
len(val_lb_data)

14961

In [12]:
total = len(train_lb_data) + len(test_lb_data) + len(val_lb_data)

In [13]:
len(train_lb_data) / total

0.6920895068804283

In [14]:
len(test_lb_data) / total

0.22814444367432463

In [15]:
len(val_lb_data) / total

0.07976604944524715

### ~ 70/8/22 train-test split
_(ignoring in-distribution splits)_

In [16]:
train_lb_data[0]

(tensor([[[0.1922, 0.1882, 0.2039,  ..., 0.2000, 0.1882, 0.1882],
          [0.1961, 0.1922, 0.2078,  ..., 0.1961, 0.1843, 0.1882],
          [0.1961, 0.1961, 0.2118,  ..., 0.1961, 0.1882, 0.1922],
          ...,
          [1.0000, 1.0000, 0.9804,  ..., 1.0000, 1.0000, 1.0000],
          [0.9922, 0.9882, 0.9961,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 0.9882, 0.9922,  ..., 1.0000, 1.0000, 1.0000]],
 
         [[0.1922, 0.1882, 0.2039,  ..., 0.2000, 0.1882, 0.1882],
          [0.1961, 0.1922, 0.2078,  ..., 0.1961, 0.1843, 0.1882],
          [0.1961, 0.1961, 0.2118,  ..., 0.1961, 0.1882, 0.1922],
          ...,
          [1.0000, 1.0000, 0.9804,  ..., 1.0000, 1.0000, 1.0000],
          [0.9922, 0.9882, 0.9961,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 0.9882, 0.9922,  ..., 1.0000, 1.0000, 1.0000]],
 
         [[0.1922, 0.1882, 0.2039,  ..., 0.2000, 0.1882, 0.1882],
          [0.1961, 0.1922, 0.2078,  ..., 0.1961, 0.1843, 0.1882],
          [0.1961, 0.1961, 0.2118,  ...,

#### _(image_data, label, metadata)_

---

In [17]:
dataset_lb.metadata_fields

['location',
 'sequence',
 'year',
 'month',
 'day',
 'hour',
 'minute',
 'second',
 'y',
 'from_source_domain']

### metadata index 0 represents location or domain ID

In [18]:
from wilds.common.grouper import CombinatorialGrouper

In [19]:
grouper = CombinatorialGrouper(dataset_lb, ['location'])

In [20]:
grouper.cardinality

tensor([323])

### 323 different domains in labeled data

In [21]:
import numpy as np
from tqdm import tqdm
from wilds.common.data_loaders import get_train_loader

In [31]:
train_lb_loader = get_train_loader("standard", train_lb_data, batch_size=128)

In [32]:
sample = next(iter(train_lb_loader))

In [33]:
sample[-1]

tensor([[  168,  5083,  2013,  ...,    40,     0,     1],
        [  177, 25908,  2013,  ...,     9,     0,     1],
        [   93, 19497,  2013,  ...,    18,     0,     1],
        ...,
        [   88, 29430,  2013,  ...,     2,    65,     1],
        [  118, 17311,  2013,  ...,    37,     1,     1],
        [  213, 13676,  2015,  ...,    22,    36,     1]])

In [34]:
sample[-1][:,0]

tensor([168, 177,  93,  93, 181, 185,  75, 102, 318, 221,  11, 259, 265,  99,
        307,  26, 186, 139, 141, 295, 118, 230, 259, 150, 162,  63, 235,   5,
        295, 259, 236,  93, 103, 158, 286, 197,  84, 296, 272, 197,  26, 162,
        117,  23, 162, 310, 262, 317,  54, 236, 194, 181, 307,  26, 265, 303,
         89, 118, 262,  11,  13, 133,   2, 253, 140, 304, 139, 235,  54,  93,
        242,  28,  26, 102, 118,  63, 286, 139, 296,  93, 235,   2, 307, 230,
         12, 165, 308, 314, 296, 202, 292,  91,  31, 265, 194, 103, 255, 205,
        243, 235, 296,  97, 307, 160, 294,  48, 118, 235, 185,  63, 197, 147,
         97,  54, 162,  63, 186, 194, 253, 265, 253, 221, 256,  54,  14,  88,
        118, 213])

In [35]:
x = []

In [36]:
c = 0
iterator = iter(train_lb_loader)
pbar = tqdm(total=100)
while c < 100:
    c += 1
    pbar.update(1)
    _, _, metadata = next(iterator)
    x.append(metadata[:,0]) # only extract the location column
pbar.close()

100%|██████████| 100/100 [06:11<00:00,  3.71s/it]


### sample 100 batches ~ 12800 examples for their domains

In [37]:
len(x)

100

In [38]:
xx = np.hstack(x)

In [39]:
xx.shape

(12800,)

In [40]:
xx

array([147, 236, 221, ..., 168,  26,  84])

In [41]:
hist, _ = np.histogram(xx, np.arange(grouper.cardinality.item()))

In [45]:
hist

array([  1,  41, 343,   0,   0, 144, 114,   0,  38,   7,   2, 196,  19,
        62,   4,  11,   6,  17,   6,  25,   3,   0,   0, 155,   0,   3,
       365,   0,   7,   0,   7,  89,  11,  49,  66,  18,   7,   5,  21,
        37,   0,   9,   2,  72,  14,  21, 198,  23,  62,   0,  10,  10,
        29,   5, 335,   1,   0,   0,   0,   0,   4,   8,   0, 244,   4,
         0,   0,   3,   1,   0,  43,   8,  28,   0,   8,  16,   0,   3,
         0,   0,   0,   0,   0,   9,  55,   8,   0,  12,  51,  18,  21,
        11,   4, 262,   6,   0,   8,  84,  10,  48,   0,   0, 220, 147,
         0,   2,  10,   1,   0,  13,  11,   3,  25,  93,   0,   0,   5,
       248, 196,   2,   0,  41,  12,  80,   6,   0,   0,   0,   2,   8,
        10,  11,   4,  40,   0,  30,   0,  19,  13, 244,  34,  71,  47,
         0,  21,   0,   0, 217,   0,  11,  16,  68,   6,   0,   0,   2,
         0, 114, 128,   5, 142,  13, 120,   0,  43,  16,  12,   8, 181,
         0,  42,   0,   3,  70,  16,   0,   0,  57,   8,  80,   

In [51]:
sum(hist == 0)

np.int64(91)

### _About 90 out of 323 domains held out from train split_